# Description
This script's purpose is to create a new image that has the model prediction label as its column value.

This is done by transforming the pixels of an image of the predicted area into an array and assigning a unique value to each array element. This unique value will be the id for that pixel. The array is then joined to the predicted csv using its unique value column and the fid column from the csv. The output of the join is then exported with the geographical information of the image so as to make it possible to display.

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import pandas as pd
from osgeo import gdal
from affine import Affine
import numpy as np
import rasterio

In [ ]:
#Reading the output from the prediction script
data1 = pd.read_csv("cashews_benin_prediction_PDL_20220217.csv")
data1.head()
csv = data1.drop(columns=['row_loc', 'col_loc','tile'])
csv_predicted_class= data1.drop(columns=['fid','row_loc', 'col_loc','tile'])
(unique, counts) = np.unique(csv_predicted_class, return_counts=True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

In [ ]:
#Open image created from Creating_IDs_for_imagepixels script
step1 = gdal.Open('large_area_fid/0_field_id.tif', gdal.GA_ReadOnly)
from PIL import Image
im = Image.open('large_area_fid/0_field_id.tif')
im = np.array(im)
im.shape

In [ ]:
#Obtaining its existing  coordinates information of the image file 
GT_input = step1.GetGeoTransform()
print(GT_input) 
transf = (399664.5859337867, 10.0, 0.0, 1100650.3206286626, 0.0, -10.0) #as obtained from GT_input
afn = Affine.from_gdal(*transf)

In [ ]:
#Reading the image and creating an array from it 
step2 = step1.GetRasterBand(1)
img_as_array = step2.ReadAsArray()
x =img_as_array
size1,size2=img_as_array.shape
output=np.zeros(shape=(size1,size2))
print("Image shape is",output.shape)

x_flatten = x.flatten()
print("Flattened image shape is",x_flatten.shape)

In [ ]:
#Creating a dataframe from the flattened image and joining it to the predicted csv table (data_1 input above)
df_x = pd.DataFrame(x_flatten, columns = ['fid'])
csv_0_merged = pd.merge(df_x, csv, on=["fid"], how='left')
csv_0_merged.head()

In [ ]:
#Keeping only the joined column and finding the frequency of each class label in the image
csv_0_merged_reduced = csv_0_merged.drop(columns=['fid'])
(unique, counts) = np.unique(csv_0_merged_reduced, return_counts=True)
frequencies_2 = np.asarray((unique, counts)).T
print(frequencies_2)

In [ ]:
#reshaping array to the original image shape
output = csv_0_merged_reduced.to_numpy()
array_output = np.reshape(output, (11021, 11125))#enter details as obtained from output.shape

In [ ]:
#Exporting the array created as a tif file
dst_crs='EPSG:32630'#coordinate system of the destination tif
output = np.int32(array_output)

In [ ]:
with rasterio.open(
    'Predicted_EPSG_32630.tif',#name of the outputfile 
    'w',
    driver='GTiff',
    height=array_output.shape[0],
    width=array_output.shape[1],
    count=1,
    dtype=np.int32,
    crs=dst_crs,
    transform=afn,
) as dest_file:
    dest_file.write(array_output, 1)
dest_file.close()